# 4. Interpolation II

Earlier, we discussed two options for the choice of interpolation basis (monomial/Vandermonde and Lagrange). Now let's take a look at the other parameter of interpolation: the choice of collocation nodes. 

We established that the error in numerical interpolation comes from two terms,

$$ ||f - \tilde{p}_n|| \leq ||f - p_n|| + ||p_n - \tilde{p}_n||,$$ 

where the former is what we'll focus on now.
To minimize errors from the second term (numerical evaluation of the interpolating polynomial), we'll use the Lagrange basis and define

$$ p(x) = p_n(x) := (L_n f)(x),$$

where $L_n$ is the Lagrange interpolating operator. Let's start with a _bad_ example: equispaced nodes on the interval $[-1, 1]$ (which we can always rescale), $x_j = -1 + \tfrac{2j}{n}$ for $j = 0, 1, \ldots, n$. In the example below, I attempt to interpolate $f(x) = \frac{1}{1 + 25x^2}$ using an increasing number of interpolation nodes. 

```{warning}
Click the rocket (launch) button on the top right of this page to interact with the plots below.
```

In [4]:
import numpy as np
import scipy.interpolate as sciint
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider
import warnings
warnings.filterwarnings('ignore') # Don't try this at home :)


# The function we wish to interpolate on [-1, 1]
f = lambda x, x0: 1/(1 + (x/x0)**2)

# We choose the collocation nodes to be equidistant 
nodes = lambda n: np.linspace(-1, 1, n)
# Try commenting the above line and uncommenting the below one 
# for Chebyshev nodes
#nodes = lambda n: np.cos(np.linspace(0, np.pi, n))

# Target nodes
x_star = np.linspace(-1, 1, 1000)

# Barycentric interpolation from given nodes
baryint = lambda nodes, n, x0, f: sciint.barycentric_interpolate(nodes(n), f(nodes(n), x0), x_star)

def draw_equiint(n, x0):
    fig, ax = plt.subplots(1, 1, figsize = (6, 6))
    ax.plot(x_star, f(x_star, x0), '--', label = '$f(x)$', color = 'black')
    ax.set_ylim(1/(1+(1/x0)**2)-0.1, 1.1)
    ax.set_xlim(-1.1, 1.1)
    ax.set_xlabel("$x$")
    ax.plot(x_star, baryint(nodes, n, x0, f), label = "$f_{"+"{}".format(n)+"}(x)$")
    ax.plot(nodes(n), f(nodes(n), x0), '.')
    ax.legend(loc = 'lower center')


interact(draw_equiint, n = IntSlider(min = 1, max = 50, value = 0),\
                     x0 = FloatSlider(min = 0.2 , max = 3, step = 0.2),\
                     continuous_update = False)            

interactive(children=(IntSlider(value=1, description='n', max=50, min=1), FloatSlider(value=0.2, description='…

<function __main__.draw_equiint(n, x0)>

```{admonition} Question
What do you observe as $n$ increases with equispaced nodes? What happens if you switch to Chebyshev nodes?
```

With increasing $n$, we see that

$$ |f - L_nf| \begin{cases}
&\to 0 \text{ as } n \to \infty \text{ in the central region,}\\
&\text{ oscillates between interpolation nodes near $\pm 1$ as } n \to \infty. 
\end{cases} $$

This is called the **Runge phenomenon**. 
If we "bunch" interpolation nodes near the edges of the interval instead, e.g. use Chebyshev nodes
$$ x_j = - \cos(j\pi/n) \quad j = 0, 1, \ldots, n,  $$
then we observe

$$ \max_{x \in [-1, 1]}| f - L_nf | \equiv || f - L_nf||_{L_\infty} \to 0 \quad \text{as } n \to \infty,$$

i.e. we see **uniform convergence**.

If you now started to change the $x_0$ parameter in the above $f(x) = \frac{1}{1 + \left(\frac{x}{x_0}\right)^2}$ from its default value $x_0 = 0.2$ to something like $x_0 = 2.0$, then even at large $n$, interpolation with equispaced nodes appears to yield an accurate answer and no Runge phenomenon is seen.

From these experiments it's clear that:
- the choice of collocation nodes has an effect on the convergence rate of interpolation,
- and so do properties of the function $f(x)$, and it turns out that for some choices of $f$, we don't have to worry about non-convergence. 

```{admonition} Question
Can you guess what property of $f(x)$ determines the convergence rate of interpolation from the above? (Hint: what did $x_0$ control?)
```
To see this, let's derive a (loose) error bound.

## 3.1 Error of interpolation for analytic $f$

```{admonition} Theorem
:class: hint
If we assume nothing about the collocation nodes, but that $f \in C^{(n+1)}[a, b]$ ($n + 1$ times continuously differentiable on the interpolation interval), then 
$$ || f - L_nf ||_{L_{\infty}} \leq \frac{||f^{(n+1)}||_{L_{\infty}}}{(n+1)!} H^{n+1}, $$
where $H$ is the length of the interpolation interval $[a, b]$, and $f^{(n+1)}$ is the $(n+1)$th derivative of $f$.
```

**Proof**: First consider the **remainder** function, defined as

$$ R_nf := f - L_nf. $$

We'll first prove that 

$$ (R_n f)(x) = \frac{f^{(n+1)}(\xi) }{(n+1)!} \prod_{j = 0}^{n} (x - x_j), $$

for $x \in [a, b]$ and some $\xi \in [a, b]$.
The expression is clearly true if $x = x_j$ (since both sides are zero), so let's assume $x$ is not one of the nodes. Then let

$$ q_{n+1}(x) := \prod_{j = 0}^n (x - x_j),$$

and consider the function 

$$ g(y) := f(y) - (L_nf )(y) - q_{n+1}(y) \frac{f(x) - (L_n f)(x)}{q_{n+1}(x)}. $$

If $f$ was $C^{n+1}$, then so is $g$, since $L_nf$ and $q_{n+1}$ are polynomials (and hence $C^{\infty}$). $g$ therefore has at least $n+2$ roots on the interpolation interval, $g'$ has at least $n+1$, and by induction, $g^{(n+1)}$ has at least one. Let that root be at $\xi$. Differentiating both sides of the expression with respect to $y$ $(n+1)$ times and evaluating at $\xi$ gives

$$ 0 = f^{(n+1)}(\xi) - 0 + (n+1)!\frac{(R_nf)(x)}{q_{n+1}(x)}, $$

where I used the definition of the remainder and that $L_nf$ is a degree-$n$ polynomial. Rearranging gives the desider expression for $R_n f$. Now using 

$$ \prod_{j = 0}^n (x - x_j) \leq (b - a)^{n+1} = H^{n+1} $$

and taking $L_{\infty}$ norms, we complete the proof.

Given an interpolation interval of length $H$, it's now clear that the factor that determines the convergence rate is $||f^{(n+1)}/(n+1)!||$...

```{admonition} Question
What is this?
```
... the magnitude of the $(n+1)$th Taylor series coefficient of $f$. Just how large can it get?

We are now going to make an even stricter assumption of $f$: instead of asking for it to be $C^{n+1}$, we'll assume it's **analytic** in a region of the complex plane.

```{admonition} Definition
:class: hint
$f$ is analytic in an open domain if at every point in the domain, it is locally given by a convergent power series. 
```

For example, if $f$ is analytic at $0$, then there exists $\rho > 0$ such that 

$$ f(z) = \sum_{n = 0}^\infty a_n z^n, \quad a_n = \frac{f^{(n)}(0)}{n!}, $$

is an an absolutely convergent series for all $z$ in the disc $|z| < \rho$, and is absolutely divergent outside of it ($|z| > \rho$), with $f$ being analytic in $|z| < \rho$.
For absolute convergence $\forall \varepsilon > 0$, in $|z| = \rho - \varepsilon$ the $a_n$ must decay as $n \to \infty$, so are bounded by some constant $c$:

$$|a_n|(\rho - \varepsilon)^n \leq c, $$

i.e. $|a_n| \leq \frac{c}{(\rho - \varepsilon)^n}$.

Let's assume $f$ is analytic in a "stadium" defined as $\{ z \in \mathbb{C}, \text{dist}(x, [a, b]) \leq R\}$, a set of points that are within a distance $R$ of the interpolation interval. Then the Taylor coefficients satisfy 

$$|a_n| \leq \frac{c}{R^n} $$

(with the expansion center being in $[a, b]$). Then 

$$ ||f - L_n f||_{L_{\infty}}  \leq \frac{c}{R^n}H^{n+1} = \tilde{c} \left( \frac{H}{R} \right)^n.$$

So for functions $f$ that are analytic in some region, it's the width of this region relative to the interpolation interval that determines the convergence rate. If it is larger than the interval width, we'll get _geometric_/_exponential_ convergence _regardless_ of the choice of nodes!

```{admonition} Question
What determines the region of analyticity of $f$ in the code experiment?
```

That region in turn is determined by the nearest singularity (pole) in $f$ (unless there is something more pathological going on), which were at $\pm i x_0$, a distance $x_0$ away from $[a, b]$.  With this in mind, revisit the experiment and play around with the $x_0$ setting.

There are, however, some good and bad news (standard results) about the convergence of interpolation and choice of nodes.

## 3.2 The good and the bad

I'll state these without proof.

```{admonition} Theorem (Faber)
:class: hint
If we constructed a sequence of interpolating operators $L_n$, each with nodes $\{ x_j^{(n)}\}_{j = 0}^n$ for $n = 0, 1, \ldots,$ then there always $\exists f \in C[a, b]$ such that $L_n f$ does not converge to $f$ uniformly on $[a, b]$.
```
On the contrary,

```{admonition} Theorem (Marcinkiewicz)
:class: hint
For each function $f \in C[a, b]$, $\exists$ a sequence of interpolating nodes $\{ x_j^{(n)}\}_{j = 0}^n$ for $n = 0, 1, \ldots,$, such that for the resulting sequence of inteprolating operators $L_n$, $L_n f$ converges to $f$ uniformly on $[a, b]$.
```

With this in mind, why is it worth clustering the interpolating nodes at the edges of the interval?

## 3.2 Why cluster nodes at the edges?

To explain this, we invoke some physics (potential theory) that will be useful later on when considering boundary intergal equations. 

Consider the **potential** function

$$\phi_{(n+1)}(z) := \frac{1}{n+1} \ln \left| \prod_{j = 0}^n (z - x_j) \right|  = - \frac{1}{n+1} \sum_{j = 0}^n \ln \frac{1}{|z - x_j|},$$

where the term in the absolute value sign on the left is none other than $q_{n+1}(z)$.

```{admonition} Question
What is this interpreted as in physics? (Hint: think electrostatics in 2D.)
```

Physically, the right-hand-side denotes the potential generated by $n+1$ charges located at each of the $x_j$, in two dimensions, $\mathbb{R}^2 = \mathbb{C}$. Each of the charges has strength $1/(n+1)$, so that they add up to $1$. Rearranging gives

$$q_{n+1}(z) = e^{(n+1)\phi_{(n+1)}(z)}. $$

As $n \to \infty$, let's say these discrete charges tend to a fixed density function $\rho(x) > 0$ on the interval $[-1, 1]$. We keep them normalized so that the total charge is still $1$,

$$ \int_{-1}^1 \rho(x)\mathrm{d}x = 1. $$

What happens to the potential they generate? It becomes 

$$ \phi_{(n+1)}(z) \to \phi(z) = - \int_{-1}^1 \rho(x) \ln \frac{1}{|z - x|} \mathrm{d} x. $$

Now identifying the charges as the interpolation nodes, we can inspect what happens at $n \to \infty$. For equidistant charges, the charge density in the limit $n \to \infty$ is uniform, i.e.

$$ \rho(x) = \frac{1}{2}, $$

(to ensure proper normalization), and (check this yourself)

\begin{align}
\phi(0) &= -1, \\
\phi(\pm 1) & -1 + \ln 2.
\end{align}

This means that $|q_{n+1}|$ is $\approx e^{(n+1) \ln 2} = 2^{n+1}$ times larger at the edges of the interval than at the center. The same applies to the basis polynomials $l_k$, which is what causes the issues associated with equispaced nodes.

Ideally, then, we seek a charge distribution $\rho(x)$, such that the generated potential $\phi(z)$ is uniform (constant) over the interpolation interval. The right distribution turns out to be

$$ \rho(x) = \frac{1}{\pi\sqrt{1 - x^2}}, $$

which is the limiting distribution of the nodes $x_j = -\cos(j \pi/n)$ for $j = 0, 1, \ldots, n$, i.e. the Chebyshev nodes. For these set of nodes, the neareast singularity in $f$ may be _arbitrarily close_ to the interval, and we still get geometric convergence.

Check it in the following experiment, showing the convergence of (barycentric) interpolation with Chebyshev nodes as the pole of $f$ moves closer to the interval:

In [5]:
# Compute n+1 Chebyshev polynomial roots
chebx = lambda n: np.cos(np.linspace(0, np.pi, n+1))

# Equidistant nodes
equinodes = lambda n: np.linspace(-1, 1, n)

def draw_convergence(x0):
    ns = np.arange(1, 200, dtype = int)
    equierr = np.array([np.max(np.abs(f(x_star, x0) - baryint(equinodes, n, x0, f))) for n in ns])
    chebyerr = np.array([np.max(np.abs(f(x_star, x0) - baryint(chebx, n, x0, f))) for n in ns])
    fig, ax = plt.subplots(1, 1, figsize = (6, 6))
    ax.semilogy(ns, equierr, label = "Equidistant")
    ax.semilogy(ns, chebyerr,'--', label = "Chebyshev")
    ax.set_ylim(1e-16, 1e0)
    ax.set_xlim(1,100)
    ax.set_ylabel("Error")
    ax.set_xlabel("$n$")
    ax.legend(loc = "lower left")
    

interact(draw_convergence, x0 = FloatSlider(min = 0.01 , max = 2, step = 0.01, value = 0.65),\
         continuous_update = False) 

interactive(children=(FloatSlider(value=0.65, description='x0', max=2.0, min=0.01, step=0.01), Output()), _dom…

<function __main__.draw_convergence(x0)>

```{admonition} Exercise
:class: danger

(Quiz 2) This question is about estimating the error in interpolation with analytic formulae and by brute force.

For a function $f$ analytic in $[-1, 1]$ and analytically continuable to the "stadium" $E_{\rho}$, where it satisfies $f \leq M$ for some $M$, [Trefethen's ATAP]() derives the following error bound on the Chebyshev interpolant (with $n+1$ nodes):
$$ ||f - p_n|| \leq \frac{4M\rho^{-n}}{\rho - 1}. $$

The stadium $E_{\rho}$ is defined as follows: if we have a circle of radius $\rho$ in the complex $z$-plane, then make the subsitution $x = (z + z^{-1})/2$, then we get the stadium $E_{\rho}$ (known as a Bernstein ellipse) in the complex $x$-plane.

- a) Show that we do indeed get an ellipse in the complex $x$-plane. What are its semi-major and semi-minor axes?
- b) Take the function $f(x) = \frac{1}{1 + (x/x_0)^2}$. What $\rho$ parameter does the Bernstein ellipse going through its poles correspond to? 
- c) Plot the expected convergence rate (ommitting anything that doesn't depend on $n$) of barycentric interpolation with respect to $n$, and the actual convergence rate, for $x_0 = 2$. What type of convergence is this? Comment on the plot.

_Hints_:
* Use an $L_{\infty}$ norm to measure the error (i.e. take the maximum over dense, equally spaced "target" points distributed on the interval).
* Use a logarithmic $y$ and linear $x$ axis (and label them!).
* Label the curves and add a legend.
```